In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.transforms import ToTensor, Compose, Grayscale, Resize, CenterCrop
import matplotlib.pyplot as plt
from PIL import Image
from parse import parse
from custom_loader import AgeDBDataset
from custom_loss_functions import AngularPenaltySMLoss
from tqdm import tqdm
import pretrainedmodels

import warnings
warnings.filterwarnings("ignore")

In [2]:
# hyper params
num_of_class = 79
hidden_unit = 256
learning_rate = 1e-04
batch_size = 64
device = torch.device("cuda")

In [3]:
dataset = AgeDBDataset(
    directory = 'cropped_face/',
    transform = Compose([
        Resize(size=(64,64)),
        CenterCrop(size=64),
        Grayscale(num_output_channels=1),
        ToTensor(),
    ]),
    device = device,
)

In [4]:
len(dataset)

15508

In [5]:
train_set, validation_set, test_set = dataset.get_loaders(
    batch_size=batch_size,
    train_size=0.8,
    test_size=0.2,
)

In [6]:
model = pretrainedmodels.resnet34(pretrained='imagenet')

In [7]:
model.conv1 = nn.Conv2d(
    in_channels=1, 
    out_channels=model.conv1.out_channels,
    kernel_size=model.conv1.kernel_size,
    stride=model.conv1.stride,
    padding=model.conv1.padding,
    bias=model.conv1.bias,
)
model.last_linear = nn.Linear(in_features=model.last_linear.in_features, out_features=num_of_class, bias=True)
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
convModel = model.to(device)

In [9]:
# Training loop
def train(model, optimizer, criterion, train_loader, num_of_epoch):
    total_step = len(train_loader)
    for epoch in range(num_of_epoch):
        for i, (imgs, labels) in (enumerate(train_loader)):
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1)%total_step == 0:
                print(f"Epoch: {epoch+1}/{num_of_epoch}, Step: {i+1}/{total_step}, Loss: {loss.item()}")

In [10]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convModel.parameters(), lr=learning_rate)

In [11]:
train(convModel, optimizer, criteria, train_set, num_of_epoch=50)

Epoch: 1/50, Step: 194/194, Loss: 4.011711597442627
Epoch: 2/50, Step: 194/194, Loss: 3.736320972442627
Epoch: 3/50, Step: 194/194, Loss: 3.3835980892181396
Epoch: 4/50, Step: 194/194, Loss: 2.572436809539795
Epoch: 5/50, Step: 194/194, Loss: 1.8246232271194458
Epoch: 6/50, Step: 194/194, Loss: 1.3095355033874512
Epoch: 7/50, Step: 194/194, Loss: 0.7669467329978943
Epoch: 8/50, Step: 194/194, Loss: 0.49597838521003723
Epoch: 9/50, Step: 194/194, Loss: 0.23186105489730835
Epoch: 10/50, Step: 194/194, Loss: 0.18123576045036316
Epoch: 11/50, Step: 194/194, Loss: 0.12190712988376617
Epoch: 12/50, Step: 194/194, Loss: 0.06537696719169617
Epoch: 13/50, Step: 194/194, Loss: 0.02215374819934368
Epoch: 14/50, Step: 194/194, Loss: 0.015214829705655575
Epoch: 15/50, Step: 194/194, Loss: 0.010412212461233139
Epoch: 16/50, Step: 194/194, Loss: 0.016510294750332832
Epoch: 17/50, Step: 194/194, Loss: 0.005272837355732918
Epoch: 18/50, Step: 194/194, Loss: 0.008008151315152645
Epoch: 19/50, Step: 194/

In [12]:
# Evaluation
def eval(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, labels in (test_loader):
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            outputs = model(imgs)
            
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            
    print(f"Accuracy: {(100*correct)/total}%")


In [13]:
eval(convModel, test_set)

Accuracy: 4.579168010319252%


In [14]:
eval(convModel, train_set)

Accuracy: 100.0%
